In [204]:
import pandas as pd 
import nltk
import numpy as np 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [205]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\santj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\santj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [206]:
df1=pd.read_csv('Combined_Jobs_Final.csv')


In [207]:
df1['Title'] = df1['Title'] + ', City: ' + df1['City'] + ', State : ' +df1['State.Name'] 

In [208]:
df1=df1[['Title', 'Position', 'City', 'State.Name', 'Job.Description']]


In [210]:
df1.dropna(subset=['City'], inplace=True)
df1.dropna(subset=['State.Name'], inplace=True)

In [ ]:
df1.shape

(83916, 5)

In [ ]:
df1.head()

,Title,Position,City,State.Name,Job.Description
0,"Server @ Tacolicious, City: Palo Alto, State :...",Server,Palo Alto,California,Tacolicious' first Palo Alto store just opened...
1,"Kitchen Staff/Chef @ Claude Lane, City: San Fr...",Kitchen Staff/Chef,San Francisco,California,\r\n\r\nNew French Brasserie in S.F. Financia...
2,"Bartender @ Machka Restaurants Corp., City: Sa...",Bartender,San Francisco,California,We are a popular Mediterranean wine bar and re...
3,"Server @ Teriyaki House, City: Brisbane, State...",Server,Brisbane,California,● Serve food/drinks to customers in a profess...
4,"Kitchen Staff/Chef @ Rosa Mexicano - Sunset, C...",Kitchen Staff/Chef,Los Angeles,California,"Located at the heart of Hollywood, we are one ..."


Now we have only included the columns which have relavent information reggarading the Job Titles

In [ ]:
df1=df1.sample(n=1000,random_state=42)

In [ ]:
df=df1
df.head()

,Title,Position,City,State.Name,Job.Description
35567,Event Coordinator (Entry Level) @ Beyond Marke...,Event Coordinator (Entry Level),Columbus,Ohio,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nEvent Coor...
16167,Psychiatric Technician - PRN - Float @ St. Luk...,Psychiatric Technician - PRN - Float,Phoenix,Arizona,Provides direct patient care in the acute care...
37311,Center Managers and Service Associates @ Ace C...,Center Managers and Service Associates,Wilmington,Delaware,ACE Cash Express is the largest and fastest gr...
42797,Sales Assistant - Great Mid-Wilshire Firm! @ O...,Sales Assistant - Great Mid-Wilshire Firm!,Los Angeles,California,Ref ID: 00320-160417Classification: Secretary/...
2970,"Macy's Seasonal Retail Fitting Room Associate,...","Macy's Seasonal Retail Fitting Room Associate,...",Los Angeles,California,Job Overview:\r\n\r\nAs a Seasonal Fitting Roo...


In [ ]:
df=df.reset_index()

In [ ]:
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def cleaning(txt):
    # Remove non-alphanumeric characters
    txt = re.sub(r'[^a-zA-Z0-9\s]', ' ', txt)
    
    # Tokenization
    tokens = nltk.word_tokenize(txt.lower())
    
    # Lemmatization instead of stemming
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stopwords.words('english')]
    
    return " ".join(lemmatized_tokens)


In [ ]:
df['State.NameT']=df['State.Name']

In [ ]:
df['Job.Description'] = df['Job.Description'].astype(str).apply(lambda x: cleaning(x))
df['Title2'] = df['Title'].astype(str).apply(lambda x: cleaning(x))

In [ ]:
df['City'] = df['City'].astype(str).apply(lambda x: cleaning(x))
df['State.Name'] = df['State.Name'].astype(str).apply(lambda x: cleaning(x))

In [ ]:
df['clean_text'] = df['Title2']+" "+df['Job.Description']+" "+df['City']+" "+df['State.Name']

In [ ]:
df['State.NameT'].value_counts().head(15)

State.NameT
California        140
Florida            72
Pennsylvania       65
Ohio               53
Texas              50
Illinois           47
Washington         46
New York           38
Minnesota          35
New Jersey         31
Michigan           28
North Carolina     27
Massachusetts      27
Missouri           25
Wisconsin          24
Name: count, dtype: int64

<h2>Vectorization 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfidf=TfidfVectorizer()
matrix=tfidf.fit_transform(df['clean_text'])
similarity = cosine_similarity(matrix)

In [ ]:
df['Position'].value_counts().head(15)

Position
Administrative Assistant                           19
Customer Service Representative                    18
Staff Accountant                                   14
Accounting Clerk                                   10
Accounts Payable Clerk                              8
Receiving/Stock Associate                           8
Accounts Receivable Clerk                           7
Customer Service / Sales ( New Grads Welcome! )     7
Full Charge Bookkeeper                              6
Bookkeeper                                          6
Retail Sales Associate / Photographer               5
Senior Accountant                                   5
Retail Sales Associate – Part-Time                  5
Data Entry Clerk                                    5
Receptionist                                        5
Name: count, dtype: int64

In [ ]:
df['Position'].value_counts().head(5)

Position
Administrative Assistant           19
Customer Service Representative    18
Staff Accountant                   14
Accounting Clerk                   10
Accounts Payable Clerk              8
Name: count, dtype: int64

- 'enumerate' function is used to create a tuple of index and similarity score 
Eg:(0,1),(1,0.533),(2,0.78)....(n,score)

- Now we use key=lambda x:x[1] because the x[0] is index and  x[1] similarity score so we arrange the Recommendations w.r.t similarity score

- Now here by 'x=df.iloc[i[0]]' gives us an index value because i is (index,similrity_score)

- and by x['Title] we will get the value of title of that index

In [ ]:
def recommendation(Position, State):
    
        # Filter indices based on the specified state
        state_indices = df[df['State.NameT'] == State].index

        indx = df[df['Position'] == Position].index[0]
        distances = sorted(list(enumerate(similarity[indx])), key=lambda x: x[1], reverse=True)

        # Get recommendations for the specified state
        state_filtered_recommendations = []
        for i in distances:
            index, similarity_score = i[0], i[1]
            if index in state_indices:
                job_title = df.iloc[index]['Title']
                state_filtered_recommendations.append((job_title))
        state_filtered_recommendations=state_filtered_recommendations[:10]

        return state_filtered_recommendations

In [ ]:
recommended_jobs = recommendation('Server','Florida')

In [ ]:
recommended_jobs

['Dining Services Associates @ Regency Park and HarborChase of Vero Beach, City: Vero Beach, State : Florida',
 'Retail Sales Associate @ Floor and Decor Inc, City: Hialeah, State : Florida',
 'Store Associate - Hiring Event - Retail Sales (Customer Service) @ ALDI, City: Casselberry, State : Florida',
 'Customer Service / Sales ( New Grads Welcome! ) @ Vector Marketing, City: Largo, State : Florida',
 'Customer Service / Sales ( New Grads Welcome! ) @ Vector Marketing, City: Fort Lauderdale, State : Florida',
 'CASHIER @ Murphy USA, City: Punta Gorda, State : Florida',
 'Marketing Assistant @ OfficeTeam, City: Jacksonville, State : Florida',
 "Connect @ Macy's Specialist, Full Time/Part Time: Sarasota, FL - Macy’s University Town Center @ Macy's, City: Sarasota, State : Florida",
 'Server @ Brookdale, City: Palm Beach Gardens, State : Florida',
 "Retail Commission Sales Associate - Impulse Trendsetter, Part Time: Orlando, FL, Macy’s The Mall at Millenia @ Macy's, City: Orlando, State 

In [ ]:
import pickle
pickle.dump(df,open('df.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
import pandas as pd
print(pd.__version__)

2.0.0
